# Alternating Pairwise Mixture DTW

This notebook implements an Alternating Pairwise Mixture DTW approach.  The only requirement in this notebook is that it implement the `offline_processing()` and `online_processing()` functions, which will be imported and run in `02_RunExperiment.ipynb`.

Our main findings:
- This approach takes much longer to compute (since we have to run several iterations) and does not lead to improved performance (very miniscule improvements).

## Offline Processing

In the offline processing stage, three things are computed and stored in the `cache/` folder:
- chroma features for the orchestra recording
- chroma features for the full mix recording
- predicted DTW alignment between the orchestra and full mix recordings

In [ ]:
import numpy as np
import librosa as lb
import os
import os.path
import import_ipynb
import align_tools
import system_utils
from hmc_mir.align import dtw, isa
from numba import jit, njit, prange
from matplotlib import pyplot as plt
from scipy.spatial.distance import cdist

In [ ]:
def offline_processing(scenario_dir, cache_dir, hop_length):
    '''
    Carries out offline processing for a simple offline DTW system.
    
    Inputs
    scenario_dir: The scenario directory to process
    cache_dir: The location of the cache directory
    hop_length: The hop length in samples used when computing chroma features
    steps: an L x 2 array specifying the allowable DTW transitions
    weights: a length L array specifying the DTW transition weights
    
    This function will store the computed chroma features and estimated alignment in the cache folder.
    '''
    
    # setup
    system_utils.verify_scenario_dir(scenario_dir)
    if os.path.exists(cache_dir):
        # print(f'{cache_dir} has already been processed.  Skipping.')
        pass
    else:
        # setup
        os.makedirs(cache_dir)

        # compute orchestra features
        o_file = f'{scenario_dir}/o.wav'
        y_o, sr = lb.core.load(o_file)
        F_o = lb.feature.chroma_cqt(y=y_o, sr=sr, hop_length=hop_length, norm=None) 

        # compute full mix features
        po_file = f'{scenario_dir}/po.wav'
        y_po, sr = lb.core.load(po_file)
        F_po = lb.feature.chroma_cqt(y=y_po, sr=sr, hop_length=hop_length, norm=None)
      
        # compute subsequence DTW alignment (orchestra as query) 
        orch_start_sec, orch_end_sec = system_utils.get_orchestra_start_end_times(scenario_dir)
        orch_start_frm = int(np.round(orch_start_sec * sr / hop_length))
        orch_end_frm = int(np.round(orch_end_sec * sr / hop_length)) + 1
        C = 1 - lb.util.normalize(F_o[:,orch_start_frm:orch_end_frm], norm=2, axis=0).T @ lb.util.normalize(F_po, norm=2, axis=0)
        wp = align_tools.compute_dtw_alignment(C, np.array([[1,1],[1,2],[2,1]]), [1,1,2], subseq = True)        
        wp[0,:] = wp[0,:] + orch_start_frm  # account for offset

        # save to cache
        np.save(f'{cache_dir}/o_chroma.npy', F_o)
        np.save(f'{cache_dir}/po_chroma.npy', F_po)
        np.save(f'{cache_dir}/o_po_align.npy', wp)
        np.save(f'{cache_dir}/orch_start_end_frm.npy', np.array([orch_start_frm, orch_end_frm]))
    
    return

In [ ]:
def verify_cache_dir(indir):
    '''
    Verifies that the specified cache directory has the required files.
    
    Inputs
    indir: The cache directory to verify
    '''
    assert os.path.exists(f'{indir}/o_chroma.npy'), f'o_chroma.npy missing from {indir}'
    assert os.path.exists(f'{indir}/po_chroma.npy'), f'po_chroma.npy missing from {indir}'
    assert os.path.exists(f'{indir}/o_po_align.npy'), f'o_po_align.npy missing from {indir}'
    assert os.path.exists(f'{indir}/orch_start_end_frm.npy'), f'orch_start_end_frm.npy missing from {indir}'

# Online Processing

In the online processing stage, we do the following:
1. compute an initial P-PO alignment using subsequence DTW with chroma features
2. Fix the P-PO alignment and re-estimate the O-PO alignment.  This is done by adding P features to the O features based on the P-PO alignment, estimating the PO - O_plus_P alignment, and then inferring the P-O alignment.
3. Fix the O-PO alignment and re-estimate the P-PO alignment.  This is done by adding O features to the P features based on the O-PO alignment, estimating the PO - P_plus_O alignment, and then inferring the P-O alignment.
4. Repeat steps 2 & 3 until convergence

Note that we save the estimated P-O alignment at each iteration so we can track performance.

In [ ]:
def online_processing(scenario_dir, out_dir, cache_dir, hop_length, steps, weights, numIters):
    '''
    Carries out `online' processing for a simple offline DTW system.
    
    Inputs
    scenario_dir: The scenario directory to process
    out_dir: The directory to put results, intermediate files, and logging info
    cache_dir: The cache directory
    hop_length: The hop length in samples used when computing chroma features
    steps: an L x 2 array specifying the allowable DTW transitions
    weights: a length L array specifying the DTW transition weights
    numIters: number of iterations to run the algorithm

    This function will compute and save the predicted alignment in the output directory in a file hyp.npy
    '''
    
    # verify & setup
    system_utils.verify_scenario_dir(scenario_dir)
    verify_cache_dir(cache_dir)
    assert not os.path.exists(out_dir), f'Output directory {out_dir} already exists.'
    os.makedirs(out_dir)
    
    # compute features
    p_file = f'{scenario_dir}/p.wav'
    y, sr = lb.core.load(p_file)
    F_p = lb.feature.chroma_cqt(y=y, sr=sr, hop_length=hop_length, norm=None)  # piano features
    F_po = np.load(f'{cache_dir}/po_chroma.npy') # full mix features
    F_o = np.load(f'{cache_dir}/o_chroma.npy') # orchestra features
    orch_start_frm, orch_end_frm = np.load(f'{cache_dir}/orch_start_end_frm.npy')
        
    # precomputed PO-O alignment
    hop_sec = hop_length / sr
    wp_BC = np.flipud(np.load(f'{cache_dir}/o_po_align.npy'))
    wp_BC = np.hstack((np.array([0,0]).reshape((2,-1)), wp_BC)) # prepend (0,0) to handle edge cases properly

    # compute P-PO alignment
    C = 1 - lb.util.normalize(F_p, norm=2, axis=0).T @ lb.util.normalize(F_po, norm=2, axis=0)
    _, _, wp_AB = dtw.dtw(C, steps, weights, True)

    # infer piano-orchestra alignment
    wp_AC = align_tools.infer_alignment(wp_AB, wp_BC, frames=True)
    np.save(f'{out_dir}/hyp0.npy', wp_AC*hop_sec)

    # re-estimation
    for i in range(numIters):

        if i%2 == 0:
            
            ### re-estimate PO-O alignment ###

            # add P features to O features according to current P-PO alignment
            F_o_mod = np.copy(F_o)
            F_p_warped = isa.time_stretch_part(F_p, F_o, wp_AC.astype(np.int64).T) # alignment path must only contain integers
            F_o_mod += F_p_warped

            # estimate PO - O_plus_P alignment
            C = 1 - lb.util.normalize(F_o_mod[:,orch_start_frm:orch_end_frm], norm=2, axis=0).T @ lb.util.normalize(F_po, norm=2, axis=0)
            wp_BC = np.flipud(align_tools.compute_dtw_alignment(C, np.array([[1,1],[1,2],[2,1]]), [1,1,2], subseq = True)) # PO - O
            wp_BC[1,:] = wp_BC[1,:] + orch_start_frm  # account for offset
            wp_BC = np.hstack((np.array([0,0]).reshape((2,-1)), wp_BC)) # prepend (0,0) to handle TSM properly

            # infer P - O alignment
            wp_AC = align_tools.infer_alignment(wp_AB, wp_BC, frames=True)
            np.save(f'{out_dir}/hyp{i+1}.npy', wp_AC*hop_sec)
        
        else:
            
            ### re-estimate P-PO alignment ###

            # add O features to P features according to current O-PO alignment
            F_p_mod = np.copy(F_p)
            F_o_warped = isa.time_stretch_part(F_o, F_p, np.flipud(wp_AC.astype(np.int64)).T)# alignment path must only contain integers
            F_p_mod += F_o_warped

            # estimate P_plus_O - PO alignment
            C = 1 - lb.util.normalize(F_p_mod, norm=2, axis=0).T @ lb.util.normalize(F_po, norm=2, axis=0)
            wp_AB = align_tools.compute_dtw_alignment(C, np.array([[1,1],[1,2],[2,1]]), [1,1,2], subseq = True) # P - PO

            # infer P - O alignment
            wp_AC = align_tools.infer_alignment(wp_AB, wp_BC, frames=True)
            np.save(f'{out_dir}/hyp{i+1}.npy', wp_AC*hop_sec)
    
    return

In [ ]:
def verify_hyp_dir(indir):
    '''
    Verifies that the specified scenario hypothesis directory has the required files.
    
    Inputs
    indir: The cache directory to verify
    '''
    assert os.path.exists(f'{indir}/hyp0.npy')

# Example


Here is an example of how to call the offline and online processing functions on a scenario directory.

In [ ]:
# scenario_dir = 'scenarios/s2'
# out_dir = 'experiments/test/s2'
# cache_dir = 'experiments/test/cache'
# hop_size = 512
# steps = np.array([1,1,1,2,2,1]).reshape((-1,2))
# weights = np.array([2,3,3])
# offline_processing(scenario_dir, cache_dir, hop_size, steps, weights)
# online_processing(scenario_dir, out_dir, cache_dir, hop_size, steps, weights)